## PySpark Joins

PySpark Join is used to combine two or more DataFrames.  
It supports all basic join type operations available in traditional SQL like:
* INNER, 
* LEFT OUTER, 
* RIGHT OUTER, 
* LEFT ANTI, 
* LEFT SEMI, 
* CROSS.

| Join String | Equivalent SQL Join |
| - | - |
| inner | INNER JOIN |
| outer, full, fullouter, full_outer | FULL OUTER JOIN |
| left, leftouter, left_outer | LEFT JOIN |
| right, rightouter, right_outer | RIGHT JOIN |
| cross	| CROSS JOIN |
| semi, leftsemi, left_semi	| EXISTS, IN |
| anti, leftanti, left_anti | NOT EXISTS, NOT IN |

In [0]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()

#### Load libraries

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField, ArrayType, MapType, DoubleType
from pyspark.sql.functions import lit, col, expr, when, sum, avg, max, min, mean, count

#### Create Spark session

In [0]:
spark = SparkSession.builder.appName('PySpark Joins').getOrCreate()

In [0]:
employees = [
  (1,'Smith',-1,'2018','10','M',3000), \
  (2,'Rose',1,'2010','20','M',4000), \
  (3,'Williams',1,'2010','10','M',1000), \
  (4,'Jones',2,'2005','10','F',2000), \
  (5,'Brown',2,'2010','40','',-1), \
  (6,'Brown',2,'2010','50','',-1) \
]

employees_columns = [
  'emp_id',
  'name',
  'superior_emp_id',
  'year_joined',
  'emp_dept_id',
  'gender',
  'salary'
]

empDF = spark.createDataFrame(data=employees, schema = employees_columns)
empDF.printSchema()
empDF.show()

root
-- emp_id: long (nullable = true)
-- name: string (nullable = true)
-- superior_emp_id: long (nullable = true)
-- year_joined: string (nullable = true)
-- emp_dept_id: string (nullable = true)
-- gender: string (nullable = true)
-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
 1| Smith| -1| 2018| 10| M| 3000|
 2| Rose| 1| 2010| 20| M| 4000|
 3|Williams| 1| 2010| 10| M| 1000|
 4| Jones| 2| 2005| 10| F| 2000|
 5| Brown| 2| 2010| 40| | -1|
 6| Brown| 2| 2010| 50| | -1|
+------+--------+---------------+-----------+-----------+------+------+

In [0]:
departments = [
  (10, 'Finance'),
  (20, 'Marketing'),
  (30, 'Sales'),
  (40, 'IT')
]

departments_columns = ['dept_id','dept_name']

deptDF = spark.createDataFrame(data=departments, schema = departments_columns)
deptDF.printSchema()
deptDF.show()

root
-- dept_id: long (nullable = true)
-- dept_name: string (nullable = true)

+-------+---------+
dept_id|dept_name|
+-------+---------+
 10| Finance|
 20|Marketing|
 30| Sales|
 40| IT|
+-------+---------+

In [0]:
genders = [
  ('M', 'Male'),
  ('F', 'Female')
]

genders_columns = ['gend_id','gend_desc']

gendDF = spark.createDataFrame(data=genders, schema = genders_columns)
gendDF.printSchema()
gendDF.show()

root
-- gend_id: string (nullable = true)
-- gend_desc: string (nullable = true)

+-------+---------+
gend_id|gend_desc|
+-------+---------+
 M| Male|
 F| Female|
+-------+---------+

#### Inner Join

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'inner'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

#### Full Outer Join

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'outer'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 null| null| null| null| null| null| null| 30| Sales|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
 6| Brown| 2| 2010| 50| | -1| null| null|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'full'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 null| null| null| null| null| null| null| 30| Sales|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
 6| Brown| 2| 2010| 50| | -1| null| null|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'fullouter'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 null| null| null| null| null| null| null| 30| Sales|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
 6| Brown| 2| 2010| 50| | -1| null| null|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

#### Left Outer Join

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'left'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
 6| Brown| 2| 2010| 50| | -1| null| null|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'leftouter'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
 6| Brown| 2| 2010| 50| | -1| null| null|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

#### Right Outer Join

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'right'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 null| null| null| null| null| null| null| 30| Sales|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

In [0]:
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'rightouter'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+-------+---------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_id|dept_name|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+
 3|Williams| 1| 2010| 10| M| 1000| 10| Finance|
 1| Smith| -1| 2018| 10| M| 3000| 10| Finance|
 4| Jones| 2| 2005| 10| F| 2000| 10| Finance|
 2| Rose| 1| 2010| 20| M| 4000| 20|Marketing|
 null| null| null| null| null| null| null| 30| Sales|
 5| Brown| 2| 2010| 40| | -1| 40| IT|
+------+--------+---------------+-----------+-----------+------+------+-------+---------+

#### Left Semi Join

In [0]:
# Is similar to INNER but doesn't return rows from right table and is more efficient
# returns only rows that matched
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'leftsemi'\
)\
.show()

+------+--------+---------------+-----------+-----------+------+------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
 1| Smith| -1| 2018| 10| M| 3000|
 4| Jones| 2| 2005| 10| F| 2000|
 3|Williams| 1| 2010| 10| M| 1000|
 2| Rose| 1| 2010| 20| M| 4000|
 5| Brown| 2| 2010| 40| | -1|
+------+--------+---------------+-----------+-----------+------+------+

#### Left Anti Join

In [0]:
# returns only rows that not matched
empDF\
.join(\
  deptDF,\
  empDF.emp_dept_id ==  deptDF.dept_id,\
  'leftanti'\
)\
.show()

+------+-----+---------------+-----------+-----------+------+------+
emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
 6|Brown| 2| 2010| 50| | -1|
+------+-----+---------------+-----------+-----------+------+------+

#### Self Join

In [0]:
#  there is no self-join type available, we can use any of the above-explained join types to join DataFrame to itself
empDF.alias('emp1').join(\
  empDF.alias('emp2'),\
  col('emp1.superior_emp_id') == col('emp2.emp_id')\
  ,'left'
)\
.select(\
  col('emp1.emp_id'),\
  col('emp1.name'),\
  col('emp2.emp_id').alias('superior_emp_id'),\
  col('emp2.name').alias('superior_emp_name')
)\
.na.fill(0)\
.na.fill('N/A')\
.show()

+------+--------+---------------+-----------------+
emp_id| name|superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
 1| Smith| 0| N/A|
 2| Rose| 1| Smith|
 3|Williams| 1| Smith|
 5| Brown| 2| Rose|
 6| Brown| 2| Rose|
 4| Jones| 2| Rose|
+------+--------+---------------+-----------------+

#### Join multiple DataFrames

In [0]:
empDF\
.join(\
  deptDF,\
  deptDF.dept_id == empDF.emp_dept_id,\
  'left'\
) \
.join(\
  gendDF,\
  gendDF.gend_id == empDF.gender,\
  'left'\
) \
.join(\
  empDF.alias('emp2'),\
  empDF.superior_emp_id == col('emp2.emp_id'),\
  'left'\
)\
.select(\
  empDF.emp_id,\
  empDF.name,\
  col('emp2.emp_id'),\
  col('emp2.name'),\
  deptDF.dept_name,\
  gendDF.gend_desc,\
  empDF.year_joined\
)\
.na.fill(0)\
.na.fill('N/A')\
.show()

+------+--------+------+-----+---------+---------+-----------+
emp_id| name|emp_id| name|dept_name|gend_desc|year_joined|
+------+--------+------+-----+---------+---------+-----------+
 1| Smith| 0| N/A| Finance| Male| 2018|
 3|Williams| 1|Smith| Finance| Male| 2010|
 2| Rose| 1|Smith|Marketing| Male| 2010|
 5| Brown| 2| Rose| IT| N/A| 2010|
 6| Brown| 2| Rose| N/A| N/A| 2010|
 4| Jones| 2| Rose| Finance| Female| 2005|
+------+--------+------+-----+---------+---------+-----------+

#### The end of the notebook